In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
from tqdm import tqdm
import random
from pprint import pprint
import re
import copy
import json

In [ ]:
from openai import OpenAI
client = OpenAI()
import time
SLEEP_TIME = 1

def gpt4(input_text, prior_messages=None):
    gpt4_kwargs = {
        "model": "gpt-4-turbo",
        "temperature": 0,
    }
    if prior_messages is None:
        messages = [{"role": "system", "content": "You are a helpful assistant."}]
    else:
        messages = copy.deepcopy(prior_messages)
    messages.append({"role": "user", "content": input_text})
    while True:
        try:
            response = client.chat.completions.create(messages=messages, **gpt4_kwargs)
            break
        except Exception as e:
            print("-"*30)
            print(e)
            if type(e).__name__ == "RateLimitError":
                print(f"Sleep for {SLEEP_TIME}......")
                time.sleep(SLEEP_TIME)
            print("-"*30)
    output_text = response.choices[0].message.content
    messages.append({"role": "assistant", "content": output_text})
    return output_text, messages

In [ ]:
questions = [
    "YOUR QUESTIONS HERE",
    "YOUR QUESTIONS HERE",
    "YOUR QUESTIONS HERE",
    "YOUR QUESTIONS HERE",
    "YOUR QUESTIONS HERE"
]

In [ ]:
prompt = """Decompose the following complex question into a collection of around 20 sub-questions that you think would be relevant to answer the complex question fully.

Organize the answer in JSON-format, shaped as {"sub-questions": ["sub-question #1", "sub-question #2", ...]}.
Place the JSON-format answer between <answer> and </answer> tags.

Complex question: $QUESTION
Answer in JSON-format:
"""

In [ ]:
subquestion_collection = list()
for q in tqdm(questions):
    sub_questions, _ = gpt4(prompt.replace("$QUESTION", q))
    try:
        sub_questions = json.loads(sub_questions.replace("<answer>", "").replace("</answer>", "").strip())
        assert "sub-questions" in sub_questions
    except:
        print(sub_questions)
    subquestion_collection.append({"question": q, "sub_questions": sub_questions["sub-questions"]})

In [ ]:
with open("decompositions.json", "w") as f:
    json.dump(subquestion_collection, f, indent=4)